<a href="https://colab.research.google.com/github/danilodtw/assistente_ia_para_professores/blob/main/assistente_de_ia_para_professores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
# -*- coding: utf-8 -*-
"""Assistente IA para Professores - Estrutura Base.ipynb

Automatically generated by Colab.

Este notebook contém a estrutura inicial para o Assistente de IA para Professores,
incluindo a instalação das bibliotecas, configuração da API Key e o menu principal.
Agora com a implementação da funcionalidade de Criação de Aulas e Conteúdo,
Preparação de Provas e Avaliações, e a implementação básica para Correção de Provas.
As ferramentas de busca no Google foi habilitada para os agentes criativos.
"""

# --- Configuração Inicial ---

# Instala as bibliotecas necessárias para usar a SDK do Google GenAI e o framework de Agentes (ADK)
# O comando -q garante uma instalação silenciosa
%pip -q install google-genai google-adk

# Importa as bibliotecas essenciais
import os
from google.colab import userdata # Para acessar segredos do Colab, como a API Key
from google import genai # SDK do Google GenAI para interagir com o modelo Gemini
from google.adk.agents import Agent # Classe base para definir agentes
from google.adk.runners import Runner # Para executar os agentes
from google.adk.sessions import InMemorySessionService # Serviço de sessão em memória (simples para este projeto)
from google.adk.tools import google_search # Ferramentas de busca do Google para agentes
from google.genai import types # Tipos para criar conteúdo (Content e Part)
import textwrap # Para formatar texto de saída
from IPython.display import display, Markdown # Para exibir texto formatado no Colab

# Configura a API Key do Google Gemini
# É recomendado armazenar sua API Key como um segredo no Colab (menu à esquerda, ícone de chave)
# com o nome 'GOOGLE_API_KEY'.
try:
    os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
    # Inicializa o cliente Gemini
    client = genai.Client()
    print("✅ API Key configurada e cliente Gemini inicializado.")
except Exception as e:
    print(f"❌ Erro ao configurar a API Key: {e}")
    print("Por favor, verifique se você armazenou a API Key como um segredo no Colab com o nome 'GOOGLE_API_KEY'.")
    # Se a API Key não estiver configurada, o código pode não funcionar corretamente.
    # Considere sair ou desabilitar funcionalidades que dependem da API.
    client = None # Garante que o cliente não seja usado se a API falhe

# Define o modelo Gemini a ser usado. Podemos ajustar isso depois se necessário.
# gemini-2.0-flash é um bom ponto de partida para velocidade e custo.
if client: # Verifica se o cliente foi inicializado com sucesso
    try:
        # Exemplo rápido para verificar se o modelo está acessível
        # list(client.models.list()) # Comentado para evitar chamada desnecessária a cada execução
        print(f"✅ Cliente Gemini acessível.")
    except Exception as e:
        print(f"❌ Erro ao acessar modelos com a API Key: {e}")
        print("Verifique se a API Key está correta e tem permissão para acessar os modelos.")
        client = None # Desabilita o cliente se os modelos não puderem ser acessados

# --- Funções Auxiliares (do material da imersão) ---

# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    """Envia uma mensagem para um agente e retorna a resposta final."""
    if client is None:
        print("🚫 Cliente Gemini não inicializado. Não é possível chamar agentes.")
        return "Erro: Serviço de IA indisponível."

    session_service = InMemorySessionService()
    # Cria uma nova sessão (IDs simples para demonstração)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera pelos eventos durante a execução do agente para capturar a resposta final
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response.strip() # Remove espaços em branco extras no final

# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  """Formata texto simples para exibição em Markdown no Colab."""
  text = text.replace('•', '  *') # Converte marcadores de lista comuns para Markdown
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True)) # Adiciona identação para melhor visualização

# --- Definição dos Agentes para Criação de Aulas ---

def agente_planejador_aula():
    """Define e retorna o Agente Planejador de Aula."""
    return Agent(
        name="agente_planejador_aula",
        model="gemini-2.0-flash", # Usando Gemini 2.0 Flash
        instruction="""
        Você é um assistente especializado em planejar estruturas de aulas para professores.
        Com base no tópico, série/nível de ensino e objetivos de aprendizagem fornecidos,
        crie uma estrutura de aula detalhada, incluindo:
        - Título da Aula
        - Público-alvo (Série/Nível)
        - Objetivos de Aprendizagem (reformulados de forma clara)
        - Duração Sugerida
        - Materiais Necessários
        - Estrutura da Aula (Dividida em seções como Introdução, Desenvolvimento, Atividades, Conclusão)
        - Sugestões de Atividades (breves descrições)
        - Pontos Chave a serem abordados em cada seção.

        Seja claro, organizado e focado em uma estrutura prática para o professor.
        Utilize a ferramenta de busca do Google para pesquisar informações e recursos relevantes sobre o tópico e os objetivos, se necessário, para enriquecer o planejamento.
        """,
        description="Agente que planeja a estrutura de aulas com base em tópico, nível e objetivos.",
        tools=[google_search] # Habilitando a ferramenta de busca do Google
    )

def agente_gerador_conteudo():
    """Define e retorna o Agente Gerador de Conteúdo da Aula."""
    return Agent(
        name="agente_gerador_conteudo",
        model="gemini-2.0-flash", # Usando Gemini 2.0 Flash
        instruction="""
        Você é um redator de conteúdo educacional, especializado em criar rascunhos de materiais de aula.
        Com base no tópico, público-alvo (série/nível) e no plano de aula fornecido,
        gere um rascunho de conteúdo para a aula.
        Siga a estrutura do plano de aula e desenvolva os pontos chave em texto explicativo.
        Utilize linguagem adequada para o público-alvo.
        Inclua exemplos, analogias ou atividades curtas diretamente no texto se apropriado.
        Não inclua o plano de aula na saída final, apenas o conteúdo gerado.
        Utilize a ferramenta de busca do Google para buscar informações adicionais ou exemplos que possam enriquecer o conteúdo da aula, se necessário.
        """,
        description="Agente que gera o rascunho do conteúdo de uma aula com base em um plano.",
        tools=[google_search] # Habilitando a ferramenta de busca do Google
    )

# --- Definição dos Agentes para Provas e Avaliações ---

def agente_gerador_questoes():
    """Define e retorna o Agente Gerador de Questões."""
    return Agent(
        name="agente_gerador_questoes",
        model="gemini-2.0-flash", # Usando Gemini 2.0 Flash
        instruction="""
        Você é um assistente especializado em gerar questões para provas e avaliações educacionais.
        Com base no tópico, série/nível de ensino, tipo de questão (múltipla escolha, dissertativa, etc.)
        e nível de dificuldade fornecidos, gere um conjunto de questões relevantes.
        Para questões de múltipla escolha, forneça as opções e a resposta correta.
        Para questões dissertativas, forneça a pergunta e, opcionalmente, pontos chave esperados na resposta.
        Formate a saída de forma clara, numerando as questões e, para múltipla escolha, listando as opções (a, b, c, d...).

        Além das questões, gere um MODELO DE CARTÃO RESPOSTA em formato de texto que possa ser facilmente preenchido pelo professor com as respostas dos alunos.
        O formato do cartão resposta deve ser:
        ---
        Questão: [Número da Questão]
        Gabarito/Critérios: [Gabarito ou Critérios da Questão]
        Resposta do Aluno: [Espaço para o professor preencher a resposta do aluno]
        ---
        Repita o bloco '--- Questão... ---' para cada questão gerada.
        Utilize a ferramenta de busca do Google para buscar informações relevantes para a criação das questões, se necessário.
        """,
        description="Agente que gera questões para provas e um modelo de cartão resposta textual.",
        tools=[google_search] # Habilitando a ferramenta de busca do Google
    )

def agente_corretor_provas():
    """Define e retorna o Agente Corretor de Provas."""
    return Agent(
        name="agente_corretor_provas",
        model="gemini-2.0-flash", # Usando Gemini 2.0 Flash (nota: correção dissertativa pode exigir modelo mais capaz)
        instruction="""
        Você é um assistente especializado em corrigir provas e avaliações.
        Você receberá o texto da prova, o gabarito/critérios e as respostas de um aluno em um formato estruturado.
        Exemplo de formato de entrada (gerado pelo agente_gerador_questoes):
        ---
        Questão: [Texto da Questão]
        Gabarito: [Gabarito]
        Resposta do Aluno: [Texto da Resposta do Aluno]
        ---
        Questão: [Texto da Questão 2]
        Gabarito/Critérios: [Texto do Gabarito ou Critérios 2]
        Resposta do Aluno: [Texto da Resposta do Aluno 2]
        ---

        Para cada bloco '--- Questão... ---' fornecido, avalie a "Resposta do Aluno" com base no "Gabarito/Critérios".
        Para múltipla escolha/verdadeiro-falso, indique se a resposta está correta ou incorreta.
        Para questões dissertativas, forneça um feedback qualitativo indicando se a resposta abordou os pontos chave do gabarito.
        Seja claro e objetivo no resultado da correção para cada questão. Apresente o resultado da correção para cada questão separadamente.
        """,
        description="Agente que corrige provas e avaliações com base em texto de entrada.",
        # tools=[google_search] # Ferramenta de busca não essencial para este agente no momento

    )


# --- Funções para as Funcionalidades ---

def criar_aulas_conteudo():
    """Função para a criação de aulas e conteúdo."""
    print("\n--- Criação de Aulas e Conteúdo ---")

    if client is None:
        print("Não é possível criar aulas. O serviço de IA não está disponível.")
        return

    # Coleta inputs do professor
    topico = input("Digite o TÓPICO da aula (ex: Revolução Francesa, Circuitos Elétricos, Funções do 1º Grau): ")
    serie_nivel = input("Digite a SÉRIE/NÍVEL dos alunos (ex: 8º Ano do Fundamental, Ensino Médio, Graduação): ")
    objetivos = input("Digite os OBJETIVOS de aprendizagem (separados por vírgula, ex: 'entender as causas', 'identificar componentes'): ")

    if not topico or not serie_nivel or not objetivos:
        print("Informações incompletas. Por favor, forneça todos os dados.")
        return

    print("\nGerando plano de aula...")
    # Instancia e chama o Agente Planejador de Aula
    planejador = agente_planejador_aula()
    # Prepara a entrada para o planejador
    entrada_planejador = f"Tópico: {topico}\nSérie/Nível: {serie_nivel}\nObjetivos: {objetivos}"
    plano_aula = call_agent(planejador, entrada_planejador)

    print("\n--- Plano de Aula Gerado ---")
    display(to_markdown(plano_aula)) # Exibe o plano formatado
    print("--------------------------")

    print("\nGerando rascunho do conteúdo da aula...")
    # Instancia e chama o Agente Gerador de Conteúdo
    gerador = agente_gerador_conteudo()
    # Prepara a entrada para o gerador, incluindo o plano gerado
    entrada_gerador = f"Tópico: {topico}\nPúblico-alvo: {serie_nivel}\nPlano de Aula:\n{plano_aula}"
    rascunho_conteudo = call_agent(gerador, entrada_gerador)

    print("\n--- Rascunho do Conteúdo da Aula Gerado ---")
    display(to_markdown(rascunho_conteudo)) # Exibe o rascunho formatado
    print("----------------------------------------")

    print("\nCriação de aulas concluída.")


def preparar_provas_avaliacoes():
    """Função para a preparação de provas e avaliações."""
    print("\n--- Preparação de Provas e Avaliações ---")

    if client is None:
        print("Não é possível preparar provas. O serviço de IA não está disponível.")
        return

    # Coleta inputs do professor
    topico = input("Digite o TÓPICO da prova/avaliação: ")
    serie_nivel = input("Digite a SÉRIE/NÍVEL dos alunos: ")
    tipo_questao = input("Digite o TIPO de questão desejado (ex: múltipla escolha, dissertativa): ")
    dificuldade = input("Digite o NÍVEL de dificuldade (ex: fácil, médio, difícil): ")
    num_questoes_str = input("Digite o NÚMERO de questões desejado: ")

    # Valida se o número de questões é um dígito
    if not num_questoes_str.isdigit():
        print("Número de questões inválido. Por favor, digite um número inteiro.")
        return
    num_questoes = int(num_questoes_str)


    if not topico or not serie_nivel or not tipo_questao or not dificuldade or num_questoes <= 0:
        print("Informações incompletas ou inválidas. Por favor, forneça todos os dados corretamente.")
        return

    print(f"\nGerando {num_questoes} questões sobre '{topico}' para {serie_nivel} ({tipo_questao}, dificuldade: {dificuldade})...")

    # Instancia e chama o Agente Gerador de Questões
    gerador_questoes = agente_gerador_questoes()
    # Prepara a entrada para o gerador
    entrada_gerador = f"Tópico: {topico}\nSérie/Nível: {serie_nivel}\nTipo de Questão: {tipo_questao}\nDificuldade: {dificuldade}\nNúmero de Questões: {num_questoes}"
    questoes_geradas = call_agent(gerador_questoes, entrada_gerador)

    print("\n--- Questões Geradas e Modelo de Cartão Resposta ---")
    display(to_markdown(questoes_geradas)) # Exibe as questões e o cartão resposta formatados
    print("--------------------------------------------------")

    print("\nPreparação de provas concluída.")


def corrigir_provas_avaliacoes():
    """Função para a correção de provas e avaliações."""
    print("\n--- Correção de Provas e Avaliações ---")

    if client is None:
        print("Não é possível corrigir provas. O serviço de IA não está disponível.")
        return

    # --- Ideias para implementação futura (avançado) ---
    # TODO: Implementar a leitura de cartões resposta digitalizados em PDF.
    # Isso envolveria:
    # 1. Processamento de imagem (ex: com OpenCV) para identificar o cartão.
    # 2. OCR (ex: com Tesseract) para ler o nome do aluno e o texto das questões/respostas dissertativas.
    # 3. Lógica para identificar as marcações nas questões de múltipla escolha.
    # 4. Comparar as respostas lidas com o gabarito (que precisaria ser fornecido em um formato processável).
    # 5. Integrar os dados extraídos com o agente_corretor_provas.
    # Nota: Esta é uma funcionalidade complexa que vai além do escopo de um MVP de imersão.
    # --- Fim das ideias para implementação futura ---


    print("\nPor favor, forneça o texto da prova, gabarito/critérios e a(s) resposta(s) do aluno no formato abaixo:")
    print("---")
    print("Questão: [Texto da Questão]")
    print("Gabarito/Critérios: [Texto do Gabarito ou Critérios]")
    print("Resposta do Aluno: [Texto da Resposta do Aluno]")
    print("--- (Use '---' para separar múltiplas questões/respuestas)")
    print("\nCole o texto completo aqui e pressione Enter duas vezes para finalizar:")

    # Coleta a entrada de texto do professor
    entrada_texto_correcao = ""
    while True:
        linha = input()
        if not linha: # Detecta linha em branco dupla para finalizar a entrada
            break
        entrada_texto_correcao += linha + "\n"

    if not entrada_texto_correcao.strip():
        print("Nenhuma entrada fornecida para correção.")
        return

    print("\nCorrigindo prova...")
    # Instancia e chama o Agente Corretor de Provas
    corretor = agente_corretor_provas()
    # A entrada para o agente é o texto coletado
    resultado_correcao = call_agent(corretor, entrada_texto_correcao.strip()) # Remove espaços em branco no início/fim

    print("\n--- Resultado da Correção ---")
    display(to_markdown(resultado_correcao)) # Exibe o resultado formatado
    print("----------------------------")

    print("\nCorreção de provas concluída.")


# --- Menu Principal ---

def menu_principal():
    """Exibe o menu principal e gerencia a interação com o usuário."""
    print("🚀 Bem-vindo(a) ao Assistente de IA para Professores 🚀")

    while True:
        print("\nPor favor, escolha uma opção:")
        print("1. Criar Aulas e Conteúdo")
        print("2. Preparar Provas e Avaliações")
        print("3. Corrigir Provas e Avaliações")
        print("4. Sair")

        escolha = input("Digite o número da opção desejada: ")

        if escolha == '1':
            criar_aulas_conteudo()
        elif escolha == '2':
            preparar_provas_avaliacoes()
        elif escolha == '3':
            corrigir_provas_avaliacoes()
        elif escolha == '4':
            print("Obrigado por usar o Assistente de IA. Até mais!")
            break # Sai do loop e encerra o programa
        else:
            print("Opção inválida. Por favor, digite um número entre 1 e 4.")

# --- Execução do Menu Principal ---

if __name__ == "__main__":
    menu_principal()


✅ API Key configurada e cliente Gemini inicializado.
✅ Cliente Gemini acessível.
🚀 Bem-vindo(a) ao Assistente de IA para Professores 🚀

Por favor, escolha uma opção:
1. Criar Aulas e Conteúdo
2. Preparar Provas e Avaliações
3. Corrigir Provas e Avaliações
4. Sair
Digite o número da opção desejada: 1

--- Criação de Aulas e Conteúdo ---
Digite o TÓPICO da aula (ex: Revolução Francesa, Circuitos Elétricos, Funções do 1º Grau): Aula de GMail
Digite a SÉRIE/NÍVEL dos alunos (ex: 8º Ano do Fundamental, Ensino Médio, Graduação): 6º ano do ensino fundamental 1
Digite os OBJETIVOS de aprendizagem (separados por vírgula, ex: 'entender as causas', 'identificar componentes'): Primeiro acesso com a conta que a escola disponibilizou para o aluno, apresentação da tela inicial do GMail, como criar um e-mail descrevendo a funcionalidade de cada item da tela de criação de e-mails, formas de envio. 

Gerando plano de aula...

--- Plano de Aula Gerado ---


> Aqui está uma sugestão de estrutura de aula detalhada sobre GMail para alunos do 6º ano, considerando os objetivos que você especificou:
> 
> 
> ## Estrutura da Aula: Introdução ao GMail (6º Ano)
> 
> **Público-alvo:** Alunos do 6º ano do Ensino Fundamental
> 
> **Objetivos de Aprendizagem:**
> 
> *   Acessar o GMail utilizando a conta fornecida pela escola.
> *   Identificar e descrever os principais elementos da tela inicial do GMail.
> *   Criar um novo e-mail, preenchendo os campos de destinatário, assunto e corpo da mensagem.
> *   Compreender e praticar as diferentes formas de envio de um e-mail.
> 
> **Duração Sugerida:** 2 aulas de 50 minutos cada (total de 100 minutos)
> 
> **Materiais Necessários:**
> 
> *   Computadores ou tablets com acesso à internet.
> *   Contas de e-mail do GMail fornecidas pela escola para cada aluno.
> *   Projetor (opcional, para demonstrações em grupo).
> *   Lousa ou quadro branco e canetas.
> *   Material de apoio impresso (opcional, com screenshots e instruções).
> 
> ### Estrutura da Aula
> 
> **Aula 1: Primeiros Passos no GMail (50 minutos)**
> 
> 1.  **Introdução (10 minutos):**
> 
>     *   Iniciar a aula perguntando aos alunos se eles já usaram e-mail antes.
>     *   Explicar o que é o GMail e para que serve (comunicação, envio de trabalhos, etc.).
>     *   Ressaltar a importância do uso responsável e seguro do e-mail.
>     *   Mencionar sobre o Family Link e a conta Google [2, 3, 5, 11, 18]
> 2.  **Acesso ao GMail (15 minutos):**
> 
>     *   Demonstrar, passo a passo, como acessar o GMail utilizando a conta da escola.
>     *   Orientar os alunos a fazerem login em seus próprios computadores/tablets.
>     *   Auxiliar individualmente aqueles que tiverem dificuldades.
> 3.  **Explorando a Tela Inicial (20 minutos):**
> 
>     *   Apresentar a tela inicial do GMail (caixa de entrada, enviados, rascunhos, etc.).
>     *   Explicar a função de cada elemento (barra de pesquisa, menu principal, etc.).
>     *   Utilizar exemplos práticos para demonstrar como encontrar e-mails.
>     *   Incentivar os alunos a explorar a interface e fazer perguntas.
> 4.  **Conclusão (5 minutos):**
> 
>     *   Revisar os principais pontos abordados na aula.
>     *   Atribuir uma tarefa simples para a próxima aula (ex: explorar as configurações).
> 
> **Aula 2: Criando e Enviando E-mails (50 minutos)**
> 
> 1.  **Revisão (5 minutos):**
> 
>     *   Relembrar os principais elementos da tela inicial do GMail.
>     *   Responder a dúvidas dos alunos sobre a tarefa da aula anterior.
> 2.  **Criando um Novo E-mail (20 minutos):**
> 
>     *   Demonstrar como criar um novo e-mail (botão "Escrever").
>     *   Explicar a função dos campos: "Para", "Assunto" e corpo da mensagem.
>     *   Dar dicas de como escrever um bom e-mail (linguagem, formatação, etc.).
> 3.  **Enviando um E-mail (15 minutos):**
> 
>     *   Explicar as diferentes formas de enviar um e-mail (botão "Enviar").
>     *   Mostrar como adicionar anexos (fotos, documentos, etc.).
>     *   Orientar os alunos a enviarem um e-mail de teste para o professor.
> 4.  **Conclusão (10 minutos):**
> 
>     *   Revisar os principais pontos abordados na aula.
>     *   Responder a dúvidas dos alunos.
>     *   Propor atividades extras para a prática do GMail.
> 
> ### Sugestões de Atividades
> 
> *   **Caça ao Tesouro no GMail:** Criar uma lista de itens para os alunos encontrarem em seus e-mails (ex: um e-mail com anexo, um e-mail de um remetente específico).
> *   **E-mail para um Amigo:** Pedir aos alunos para escreverem um e-mail para um amigo ou familiar, contando sobre o que aprenderam na aula.
> *   **Debate Online:** Criar um fórum de discussão no GMail para os alunos debaterem um tema relevante.
> *   **Criação de um Mural Virtual:** Utilizar o GMail para os alunos enviarem fotos e mensagens para criar um mural virtual da turma.
> *   **Vídeos do Youtube:** Apresentar vídeos do youtube para auxiliar no aprendizado. [1, 6, 7, 8, 20, 21]
> 
> ### Pontos-Chave a serem Abordados
> 
> *   **Introdução:** O que é e para que serve o GMail, importância do uso responsável.
> *   **Acesso:** Como fazer login com a conta da escola, segurança da senha.
> *   **Tela Inicial:** Identificação e função dos principais elementos.
> *   **Novo E-mail:** Criação, preenchimento dos campos, dicas de escrita.
> *   **Envio:** Formas de envio, anexos, confirmação de envio.
> 
> Lembre-se de adaptar a estrutura e as atividades à realidade da sua turma e aos recursos disponíveis na escola. É fundamental criar um ambiente de aprendizado divertido e interativo, incentivando os alunos a explorarem o GMail e a utilizarem essa ferramenta de forma consciente e produtiva.

--------------------------

Gerando rascunho do conteúdo da aula...

--- Rascunho do Conteúdo da Aula Gerado ---


> Tudo bem! Aqui está uma sugestão de rascunho de conteúdo para sua aula de GMail para alunos do 6º ano:
> 
> ## Aula 1: Primeiros Passos no GMail
> 
> Olá, pessoal! Tudo bem com vocês? Hoje, vamos começar a aprender sobre uma ferramenta muito legal e útil: o GMail!
> 
> **1. Introdução (10 minutos): O que é GMail?**
> 
> Alguém aqui já ouviu falar em e-mail? (Espere pelas respostas). Isso mesmo! E-mail é como uma carta, só que em vez de papel e caneta, a gente usa o computador ou o celular para escrever e enviar. O GMail é um tipo de e-mail, só que feito pelo Google. É como se fosse o carteiro do mundo digital!
> 
> O GMail serve para muitas coisas:
> 
> *   **Comunicar:** Conversar com amigos, familiares e professores.
> *   **Enviar trabalhos:** Mandar as tarefas da escola para o professor.
> *   **Receber notícias:** Assinar revistas e jornais online.
> *   **Compartilhar ideias:** Participar de grupos de discussão.
> 
> Mas, assim como no mundo real, no mundo digital também precisamos ter cuidado. É muito importante usar o GMail de forma responsável e segura. Combinado?
> 
> Ah! E um detalhe importante: como vocês são pequenos, seus pais podem usar um aplicativo chamado Family Link para ajudar a proteger sua conta do Google. É como se eles pudessem dar uma olhadinha para garantir que está tudo bem, sabe?
> 
> **2. Acesso ao GMail (15 minutos): Vamos Entrar no GMail!**
> 
> Agora, vamos aprender como entrar no GMail usando a conta da escola. Prestem bastante atenção!
> 
> 1.  **Liguem o computador ou tablet.**
> 2.  **Abram o navegador de internet** (Chrome, Firefox, Safari...). É aquele programa que a gente usa para entrar nos sites.
> 3.  **Escrevam** [gmail.com](http://gmail.com) **na barra de endereço** (onde a gente escreve o nome dos sites) e apertem "Enter".
> 4.  Vai aparecer uma tela pedindo o seu e-mail e senha. **Peçam ajuda ao professor** para colocar o e-mail e a senha que a escola deu para vocês.
> 5.  Pronto! Vocês estão dentro do GMail!
> 
> Se alguém tiver dificuldade, não se preocupe! O professor vai ajudar cada um de vocês.
> 
> **3. Explorando a Tela Inicial (20 minutos): Conhecendo o GMail por Dentro**
> 
> Uau! Quanta coisa diferente nessa tela, né? Mas calma, não se assustem! Vamos conhecer cada pedacinho do GMail:
> 
> *   **Caixa de Entrada:** É onde chegam todos os e-mails que as pessoas mandam para você. É como se fosse a caixa de correio da sua casa.
> *   **Enviados:** Aqui ficam guardados todos os e-mails que você já mandou para alguém.
> *   **Rascunhos:** Sabe quando a gente começa a escrever uma carta, mas não termina? Então, ela fica guardada aqui, para a gente terminar depois.
> *   **Barra de Pesquisa:** Se você quiser encontrar um e-mail antigo, é só escrever o nome da pessoa ou o assunto aqui. É como um detetive de e-mails!
> *   **Menu Principal:** É um botão que tem três risquinhos. Quando a gente clica nele, aparecem outras opções, como "Spam" (que são e-mails de propaganda) e "Lixeira" (onde vão os e-mails que a gente não quer mais).
> 
> Que tal explorarmos um pouco? Tentem encontrar a Caixa de Entrada, os Enviados e a Barra de Pesquisa. Se tiverem alguma dúvida, é só perguntar!
> 
> **4. Conclusão (5 minutos): O Que Aprendemos Hoje?**
> 
> Hoje, aprendemos o que é o GMail, para que ele serve e como entrar na nossa conta da escola. Também conhecemos os principais pedacinhos da tela inicial do GMail.
> 
> Para a próxima aula, quero que vocês explorem um pouco mais o GMail. Cliquem nos botões, leiam os e-mails (com a ajuda do professor, se precisarem) e tentem descobrir para que serve cada coisa. Combinado?
> 
> ## Aula 2: Criando e Enviando E-mails
> 
> Olá, pessoal! Preparados para a segunda parte da nossa aventura no mundo do GMail?
> 
> **1. Revisão (5 minutos): Recapitulando**
> 
> Na aula passada, aprendemos a entrar no GMail e a conhecer a tela inicial. Alguém lembra o que é a Caixa de Entrada? E os Enviados? (Espere pelas respostas).
> 
> Muito bem! Vocês estão craques!
> 
> Alguém teve alguma dúvida sobre a tarefa de casa? (Responda às dúvidas).
> 
> **2. Criando um Novo E-mail (20 minutos): Escrevendo Nossa Mensagem**
> 
> Agora, vamos aprender a escrever um e-mail! É muito fácil!
> 
> 1.  **Clique no botão "Escrever".** Ele geralmente fica no canto superior esquerdo da tela e tem um símbolo de "+".
> 2.  Vai abrir uma janela com um monte de espaços para preencher. Vamos ver cada um deles:
>     *   **Para:** Aqui, você escreve o endereço de e-mail da pessoa para quem você quer mandar a mensagem. É como se fosse o endereço da casa da pessoa.
>     *   **Assunto:** Aqui, você escreve sobre o que é o e-mail. É como se fosse o título da carta. Por exemplo: "Tarefa de Matemática" ou "Convite para o Cinema".
>     *   **Corpo da Mensagem:** Aqui, você escreve tudo o que você quer falar. É como se fosse o texto da carta.
> 
> Dicas para escrever um bom e-mail:
> 
> *   Use uma linguagem educada e respeitosa.
> *   Escreva frases curtas e fáceis de entender.
> *   Use emojis para deixar a mensagem mais divertida (mas não exagere!).
> *   Revise o texto antes de enviar para não ter erros de português.
> 
> **3. Enviando um E-mail (15 minutos): Mandando a Mensagem para o Mundo**
> 
> Depois de escrever o e-mail, é hora de enviar!
> 
> 1.  **Clique no botão "Enviar".** Ele geralmente fica embaixo da mensagem e tem um símbolo de um aviãozinho de papel.
> 2.  Se você quiser mandar uma foto ou um documento junto com o e-mail, clique no botão "Anexar". É como se você estivesse colocando um presente dentro da carta.
> 3.  Antes de enviar, confira se o endereço de e-mail da pessoa está correto e se você escreveu tudo o que queria.
> 4.  Pronto! É só clicar em "Enviar" e esperar a pessoa responder.
> 
> Hoje, vamos enviar um e-mail de teste para o professor. Assim, vocês aprendem a usar o GMail e o professor vê se vocês estão aprendendo direitinho.
> 
> **4. Conclusão (10 minutos): Parabéns, Pequenos Escritores!**
> 
> Hoje, aprendemos a criar e enviar e-mails! Vocês viram como é fácil?
> 
> Agora, vocês podem usar o GMail para conversar com amigos, mandar trabalhos para a escola e muito mais.
> 
> Para praticar, que tal escreverem um e-mail para um amigo ou familiar, contando sobre o que vocês aprenderam na aula de hoje?
> 
> Espero que vocês tenham gostado da aula de GMail! Até a próxima!

----------------------------------------

Criação de aulas concluída.

Por favor, escolha uma opção:
1. Criar Aulas e Conteúdo
2. Preparar Provas e Avaliações
3. Corrigir Provas e Avaliações
4. Sair
Digite o número da opção desejada: 2

--- Preparação de Provas e Avaliações ---
Digite o TÓPICO da prova/avaliação: Aula de GMail
Digite a SÉRIE/NÍVEL dos alunos: 6º ano do ensino fundamental 1
Digite o TIPO de questão desejado (ex: múltipla escolha, dissertativa): apenas múltipla escolha
Digite o NÍVEL de dificuldade (ex: fácil, médio, difícil): Médio 5 opções
Digite o NÚMERO de questões desejado: 12

Gerando 12 questões sobre 'Aula de GMail' para 6º ano do ensino fundamental 1 (apenas múltipla escolha, dificuldade: Médio 5 opções)...

--- Questões Geradas e Modelo de Cartão Resposta ---


> Ok, vou gerar 12 questões de múltipla escolha sobre o tema "Aula de Gmail" para alunos do 6º ano do ensino fundamental, com nível de dificuldade médio e 5 opções de resposta. Além disso, criarei um modelo de cartão resposta textual.
> 
> 
> ## Questões de Múltipla Escolha - Aula de Gmail (6º ano)
> 
> 1.  **O que é o Gmail?**
>     *   a) Um jogo online.
>     *   b) Um programa de edição de vídeos.
>     *   c) Um serviço de e-mail gratuito do Google.
>     *   d) Uma rede social para compartilhar fotos.
>     *   e) Um aplicativo para fazer compras online.
>     **Resposta correta: c**
> 
> 2.  **Para que serve o Gmail?**
>     *   a) Apenas para enviar mensagens para si mesmo.
>     *   b) Para enviar e receber mensagens eletrônicas (e-mails).
>     *   c) Para assistir vídeos engraçados.
>     *   d) Para criar desenhos e pinturas.
>     *   e) Para jogar jogos educativos.
>     **Resposta correta: b**
> 
> 3.  **Qual o primeiro passo para usar o Gmail?**
>     *   a) Comprar um computador novo.
>     *   b) Aprender a programar.
>     *   c) Criar uma conta no Gmail (Conta Google).
>     *   d) Instalar um programa no computador.
>     *   e) Pedir permissão para um adulto.
>     **Resposta correta: c**
> 
> 4.  **O que é um "e-mail"?**
>     *   a) Uma carta escrita à mão.
>     *   b) Uma mensagem enviada pela internet.
>     *   c) Um programa de computador antigo.
>     *   d) Um tipo de doce muito gostoso.
>     *   e) Uma forma de comunicação apenas para empresas.
>     **Resposta correta: b**
> 
> 5.  **Qual das opções abaixo NÃO faz parte da tela principal do Gmail?**
>     *   a) Caixa de entrada.
>     *   b) Enviar.
>     *   c) Lixeira.
>     *   d) Configurações do computador.
>     *   e) Spam.
>     **Resposta correta: d**
> 
> 6.  **O que significa a palavra "Spam" no Gmail?**
>     *   a) Mensagens importantes de amigos.
>     *   b) Mensagens que você já leu.
>     *   c) Mensagens que foram apagadas.
>     *   d) Mensagens de propaganda ou mensagens indesejadas.
>     *   e) Mensagens com vírus.
>     **Resposta correta: d**
> 
> 7.  **Como você envia um e-mail para alguém?**
>     *   a) Jogando o computador pela janela.
>     *   b) Digitando o endereço de e-mail do destinatário, escrevendo a mensagem e clicando em "Enviar".
>     *   c) Desenhando a mensagem em um papel e enviando pelo correio.
>     *   d) Gritando a mensagem bem alto.
>     *   e) Esperando que a pessoa adivinhe a mensagem.
>     **Resposta correta: b**
> 
> 8.  **O que é a "Caixa de Entrada" do Gmail?**
>     *   a) O lugar onde você guarda seus brinquedos.
>     *   b) O lugar onde você recebe os e-mails que as pessoas te enviam.
>     *   c) O lugar onde você escreve seus e-mails.
>     *   d) O lugar onde você guarda seus documentos da escola.
>     *   e) O lugar onde você encontra jogos online.
>     **Resposta correta: b**
> 
> 9.  **É seguro compartilhar sua senha do Gmail com seus amigos?**
>     *   a) Sim, todos os amigos são confiáveis.
>     *   b) Sim, para que eles possam te ajudar a usar o Gmail.
>     *   c) Não, sua senha é pessoal e você não deve compartilhá-la com ninguém, exceto seus pais ou responsáveis.
>     *   d) Depende, se o amigo prometer não contar para ninguém.
>     *   e) Talvez, se você esquecer a sua senha.
>     **Resposta correta: c**
> 
> 10. **O que é um "anexo" em um e-mail?**
>     *   a) Uma foto sua.
>     *   b) Um clipe de papel.
>     *   c) Um arquivo (foto, vídeo, documento) que você envia junto com o e-mail.
>     *   d) Um adesivo engraçado.
>     *   e) Uma parte da mensagem principal.
>     **Resposta correta: c**
> 
> 11. **Qual botão você deve clicar para começar a escrever um novo e-mail no Gmail?**
>     *   a) Lixeira.
>     *   b) Spam.
>     *   c) Escrever.
>     *   d) Recebidos.
>     *   e) Arquivar.
>     **Resposta correta: c**
> 
> 12. **Se você receber um e-mail de alguém que você não conhece, pedindo informações pessoais, o que você deve fazer?**
>     *   a) Responder o e-mail com todas as informações.
>     *   b) Ignorar o e-mail e contar para um adulto (pais, responsáveis ou professor).
>     *   c) Enviar um vídeo engraçado para a pessoa.
>     *   d) Marcar o e-mail como importante.
>     *   e) Compartilhar o e-mail com seus amigos.
>     **Resposta correta: b**
> 
> ## MODELO DE CARTÃO RESPOSTA
> 
> 
> ```
> ---
> Questão: 1
> Gabarito/Critérios: c
> Resposta do Aluno:
> ---
> Questão: 2
> Gabarito/Critérios: b
> Resposta do Aluno:
> ---
> Questão: 3
> Gabarito/Critérios: c
> Resposta do Aluno:
> ---
> Questão: 4
> Gabarito/Critérios: b
> Resposta do Aluno:
> ---
> Questão: 5
> Gabarito/Critérios: d
> Resposta do Aluno:
> ---
> Questão: 6
> Gabarito/Critérios: d
> Resposta do Aluno:
> ---
> Questão: 7
> Gabarito/Critérios: b
> Resposta do Aluno:
> ---
> Questão: 8
> Gabarito/Critérios: b
> Resposta do Aluno:
> ---
> Questão: 9
> Gabarito/Critérios: c
> Resposta do Aluno:
> ---
> Questão: 10
> Gabarito/Critérios: c
> Resposta do Aluno:
> ---
> Questão: 11
> Gabarito/Critérios: c
> Resposta do Aluno:
> ---
> Questão: 12
> Gabarito/Critérios: b
> Resposta do Aluno:
> ---
> 
> ```

--------------------------------------------------

Preparação de provas concluída.

Por favor, escolha uma opção:
1. Criar Aulas e Conteúdo
2. Preparar Provas e Avaliações
3. Corrigir Provas e Avaliações
4. Sair
Digite o número da opção desejada: 4
Obrigado por usar o Assistente de IA. Até mais!
